In [2]:
import pandas as pd
import numpy as np
from scipy.signal import butter, lfilter

In [3]:
# Cargar el archivo CSV proporcionado
file = 'C:/Users/HP/OneDrive/Escritorio/TFG/Psychopy/Yoli_2cancion_popular.csv'
df = pd.read_csv(file)

# Eliminar los datos del Channel 5 (última columna)
df_filtered = df.iloc[:, :-1] #Fichero sin el último canal


In [4]:
#Función para crear un filtro paso-alto
def butter_highpass(cutoff, fs, order=5):
    nyq = 0.5 * fs
    normal_cutoff = cutoff / nyq
    b, a = butter(order, normal_cutoff, btype='high', analog=False)
    return b, a

# Función para aplicar el filtro a los datos
def highpass_filter(data, cutoff=4.0, fs=256.0, order=5):
    b, a = butter_highpass(cutoff, fs, order=order)
    y = lfilter(b, a, data)
    return y

In [5]:
#Función para crear un filtro paso banda para ir filtrando las bandas de las ondas
# Función para crear un filtro pasa-banda
def butter_bandpass(baja_frec, alta_frec, fs, order=4):
    nyq = 0.5 * fs
    baja = baja_frec / nyq
    alta = alta_frec / nyq
    b, a = butter(order, [baja, alta], btype='band')
    return b, a
# Función para aplicar un filtro pasa-banda
def bandpass_filter(data, baja_frec, alta_frec, fs, order=4):
    b, a = butter_bandpass(baja_frec, alta_frec, fs, order=order)
    y = lfilter(b, a, data)
    return y

In [6]:
#Aplico el filtro al fichero
df_delta = df_filtered.copy()
for column in df_delta.columns[1:]:
    df_delta[column] = highpass_filter(df_delta[column].values)

In [7]:
# Parámetros de frecuencia para las diferentes bandas
fs = 256  # Frecuencia de muestreo
bands = {
    'Theta': (4, 8),
    'Alpha': (8, 12),
    'Beta': (12, 30),
    'Gamma': (30, 100)
}

# Parámetros para la segmentación
num_frames = 14
overlap_ratio = 0.5


In [8]:
# Lista para almacenar los vectores de características de cada frame 1x80
feature_vectors = []

# Dividir el DataFrame en 14 frames con solapamiento del 50%
total_samples = len(df_delta)
frame_size = total_samples // (num_frames + (num_frames - 1) * overlap_ratio) #tamaño de cada frame
frame_size = int(frame_size)
overlap = int(frame_size * overlap_ratio)

# Iterar sobre los 14 frames
for i in range(num_frames):
    start_index = i * (frame_size - overlap)
    end_index = start_index + frame_size
    
    # Crear el vector para almacenar las características
    feature_vector = []
    
    # Iterar sobre cada canal (columna) en el DataFrame (excluyendo la primera columna:timestamp)
    for channel in df_delta.columns[1:]:
        frame = df_delta[channel].iloc[start_index:end_index].copy()

        # Procesar las ondas Theta, Alpha, Beta, Gamma y la señal EEG en bruto (raw EEG)
        for band_name, (baja_frec, alta_frec) in bands.items():
            filtered_data = bandpass_filter(frame.values, baja_frec, alta_frec, fs)
            
            # Calcular características
            feature_vector.append(filtered_data.min())
            feature_vector.append(filtered_data.max())
            feature_vector.append(filtered_data.mean())
            feature_vector.append(((filtered_data[:-1] * filtered_data[1:]) < 0).sum())  # ZCR
    
        # Añadir las características de la señal EEG en bruto
        feature_vector.append(frame.min())
        feature_vector.append(frame.max())
        feature_vector.append(frame.mean())
        feature_vector.append(((frame[:-1].values * frame[1:].values) < 0).sum())  # ZCR
    
    # Añadir el vector de características a la lista
    feature_vectors.append(feature_vector)

# Convertir la lista de vectores de características en un DataFrame para su análisis
df_features = pd.DataFrame(feature_vectors, columns=[f'feature_{i+1}' for i in range(80)])

# Guardar el DataFrame de características en un archivo CSV si es necesario
df_features.to_csv('C:/Users/HP/OneDrive/Escritorio/TFG/Preprocesado/feature_Yoli_2_pop.csv', index=False)

print("Proceso completado y las características se han guardado en 'feature_vectors.csv'")


-149.11834437410732
-443.6062479285595
-264.0810903161566
-732.2627078482701


KeyError: 0